In [17]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# بارگذاری دیتاست Iris
data = load_iris()
X = data.data        # ویژگی‌ها
y = data.target      # برچسب‌ها (0, 1, 2)

# تقسیم داده به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# تعریف و آموزش شبکه عصبی MLP
mlp = MLPClassifier(hidden_layer_sizes=(10,),
                    activation='relu',
                    solver='adam',
                    max_iter=300,
                    random_state=42)
mlp.fit(X_train, y_train)

# پیش‌بینی روی داده‌های تست و محاسبه دقت
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("دقت شبکه عصبی استاندارد:", accuracy)


دقت شبکه عصبی استاندارد: 0.7333333333333333


/home/pydevcasts/Templates/MLHub/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
! pip install pygad


In [20]:
import numpy as np
import pygad
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# تعریف توابع فعالسازی
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    # پیاده‌سازی softmax به صورت عددی پایدار
    exp_scores = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

# تابع پیش‌انتشار شبکه عصبی (دو لایه: یک لایه پنهان و خروجی)
def forward_propagation(X, solution):
    # تعیین ابعاد (ورودی، پنهان، خروجی)
    input_dim = 4    # ۴ ویژگی دیتاست Iris
    hidden_dim = 10  # تعداد نورون‌های لایه پنهان
    output_dim = 3   # تعداد کلاس‌ها (خروجی)
    
    # استخراج وزن‌ها و بایاس‌ها از بردار حل (solution)
    # وزن‌های ورودی به پنهان: از اندیس 0 تا 4*10
    W1 = solution[0: input_dim * hidden_dim].reshape((input_dim, hidden_dim))
    # بایاس لایه پنهان: اندیس بعدی به اندازه hidden_dim
    b1 = solution[input_dim * hidden_dim: input_dim * hidden_dim + hidden_dim].reshape((hidden_dim,))
    # وزن‌های لایه پنهان به خروجی: بعد از بایاس پنهان، به اندازه hidden_dim*output_dim
    W2 = solution[input_dim * hidden_dim + hidden_dim: input_dim * hidden_dim + hidden_dim + hidden_dim * output_dim].reshape((hidden_dim, output_dim))
    # بایاس خروجی: باقی‌مانده
    b2 = solution[input_dim * hidden_dim + hidden_dim + hidden_dim * output_dim:].reshape((output_dim,))
    
    # محاسبه خروجی لایه پنهان
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    
    # محاسبه خروجی نهایی با استفاده از softmax
    Z2 = np.dot(A1, W2) + b2
    output = softmax(Z2)
    return output

# بارگذاری دیتاست Iris و تقسیم به آموزش و تست
data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# اصلاح تابع تناسب (Fitness Function)
def fitness_func(ga_instance, solution, solution_idx):
    predictions_prob = forward_propagation(X_train, solution)
    predictions = np.argmax(predictions_prob, axis=1)
    accuracy = np.mean(predictions == y_train)
    return accuracy  # دقت بالاتر بهتر است.

# تعیین تعداد ژن‌ها (کل تعداد پارامترها در شبکه)
input_dim = 4
hidden_dim = 10
output_dim = 3
num_genes = input_dim * hidden_dim + hidden_dim + hidden_dim * output_dim + output_dim  # 4*10 + 10 + 10*3 + 3 = 83

# تنظیمات الگوریتم ژنتیک
ga_instance = pygad.GA(num_generations=50,
                       num_parents_mating=10,
                       fitness_func=fitness_func,
                       sol_per_pop=20,
                       num_genes=num_genes,
                       mutation_percent_genes=10,
                       mutation_type="random",
                       random_mutation_min_val=-1,
                       random_mutation_max_val=1)

# اجرای الگوریتم ژنتیک
ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("بهترین میزان تناسب (دقت) به دست آمده در آموزش:", solution_fitness)

# ارزیابی شبکه با وزن‌های بهینه شده روی داده‌های تست
predictions_prob_test = forward_propagation(X_test, solution)
predictions_test = np.argmax(predictions_prob_test, axis=1)
accuracy_test = np.mean(predictions_test == y_test)
print("دقت روی مجموعه تست با وزن‌های بهینه شده توسط GA:", accuracy_test)


بهترین میزان تناسب (دقت) به دست آمده در آموزش: 0.9833333333333333
دقت روی مجموعه تست با وزن‌های بهینه شده توسط GA: 0.9333333333333333


In [16]:
%% Optimizing MLP Using Genetic Algorithm in Octave

% 1. Load Iris dataset
filename = 'upload/iris.csv'; % Ensure iris.csv is in your working directory
fileID = fopen(filename, 'r');
if fileID == -1
    error('Error loading the dataset. Ensure the file "%s" exists.', filename);
end

data = textscan(fileID, '%f %f %f %f %s', 'Delimiter', ',', 'HeaderLines', 0);
fclose(fileID);

inputs = [data{1}, data{2}, data{3}, data{4}]'; % Features
species = data{5};                              % Labels

% Convert labels to one-hot format
unique_species = unique(species);
num_samples = numel(species);
targets = zeros(numel(unique_species), num_samples);
for i = 1:num_samples
    idx = find(strcmp(unique_species, species{i}));
    targets(idx, i) = 1;
end

% 2. Define MLP structure
input_dim = size(inputs, 1);
hidden_dim = 10;
output_dim = size(targets, 1);

% Forward propagation function
function [output, hidden] = forward(X, W1, b1, W2, b2)
    hidden = max(0, W1 * X + b1); % ReLU activation
    output = softmax(W2 * hidden + b2);
end

% Softmax function
function output = softmax(x)
    exp_x = exp(x - max(x, [], 1)); % Stability improvement
    output = exp_x ./ sum(exp_x, 1);
end

% Cost function
function fitness = mlp_cost(solution, inputs, targets)
    input_dim = size(inputs, 1);
    hidden_dim = 10;
    output_dim = size(targets, 1);

    W1 = reshape(solution(1:hidden_dim*input_dim), hidden_dim, input_dim);
    b1 = reshape(solution(hidden_dim*input_dim+1:hidden_dim*input_dim+hidden_dim), hidden_dim, 1);
    W2 = reshape(solution(hidden_dim*input_dim+hidden_dim+1:hidden_dim*input_dim+hidden_dim+hidden_dim*output_dim), output_dim, hidden_dim);
    b2 = reshape(solution(hidden_dim*input_dim+hidden_dim+hidden_dim*output_dim+1:end), output_dim, 1);

    [outputs, ~] = forward(inputs, W1, b1, W2, b2);

    [~, predicted_labels] = max(outputs, [], 1);
    [~, true_labels] = max(targets, [], 1);
    accuracy = sum(predicted_labels == true_labels) / numel(true_labels);

    % Handle invalid accuracy cases
    if isnan(accuracy) || isinf(accuracy)
        fitness = Inf; % Penalize invalid solutions
    else
        fitness = -accuracy; % GA minimizes the cost
    end
end

% 3. Genetic Algorithm implementation
num_genes = hidden_dim * input_dim + hidden_dim + hidden_dim * output_dim + output_dim;
population_size = 50;
num_generations = 50;
mutation_rate = 0.01;

population = rand(population_size, num_genes) * 2 - 1; % Initialize population
overall_best_fitness = Inf;
optimal_solution = [];

for generation = 1:num_generations
    fitness_scores = zeros(population_size, 1);
    for i = 1:population_size
        fitness_scores(i) = mlp_cost(population(i, :), inputs, targets);
    end

    % Avoid invalid fitness values
    valid_indices = ~isinf(fitness_scores);
    if ~any(valid_indices)
        fprintf('All solutions invalid at generation %d. Exiting.\n', generation);
        break;
    end

    % Select parents (tournament selection)
    tournament_size = 3;
    parents = zeros(population_size, num_genes);
    for i = 1:population_size
        tournament_indices = randi(population_size, 1, tournament_size);
        [~, winner] = min(fitness_scores(tournament_indices));
        parents(i, :) = population(tournament_indices(winner), :);
    end

    % Crossover (uniform crossover)
    offspring = zeros(population_size, num_genes);
    for i = 1:2:population_size
        mask = randi([0, 1], 1, num_genes);
        offspring(i, :) = parents(i, :) .* mask + parents(i+1, :) .* (1 - mask);
        offspring(i+1, :) = parents(i+1, :) .* mask + parents(i, :) .* (1 - mask);
    end

    % Mutation
    mutation_mask = rand(population_size, num_genes) < mutation_rate;
    offspring(mutation_mask) = offspring(mutation_mask) + randn(sum(mutation_mask(:)), 1) * 0.1;

    population = offspring;

    % Track best solution
    [best_fitness, best_idx] = min(fitness_scores(valid_indices));
    if best_fitness < overall_best_fitness
        overall_best_fitness = best_fitness;
        optimal_solution = population(valid_indices(best_idx), :);
    end

    fprintf('Generation %d, Best Fitness: %.4f\n', generation, -overall_best_fitness);
end

% Handle cases where no valid solution was found
if isempty(optimal_solution)
    error('No valid solution found. Check data or algorithm settings.');
end

% Extract optimized weights and biases
W1_opt = reshape(optimal_solution(1:hidden_dim*input_dim), hidden_dim, input_dim);
b1_opt = reshape(optimal_solution(hidden_dim*input_dim+1:hidden_dim*input_dim+hidden_dim), hidden_dim, 1);
W2_opt = reshape(optimal_solution(hidden_dim*input_dim+hidden_dim+1:hidden_dim*input_dim+hidden_dim+hidden_dim*output_dim), output_dim, hidden_dim);
b2_opt = reshape(optimal_solution(hidden_dim*input_dim+hidden_dim+hidden_dim*output_dim+1:end), output_dim, 1);

% Evaluate optimized network
[outputs_opt, ~] = forward(inputs, W1_opt, b1_opt, W2_opt, b2_opt);
[~, predicted_labels_opt] = max(outputs_opt, [], 1);
[~, true_labels_opt] = max(targets, [], 1);
accuracy_opt = sum(predicted_labels_opt == true_labels_opt) / numel(true_labels_opt);
fprintf('Final Optimized Accuracy: %.2f%%\n', accuracy_opt * 100);


UsageError: Cell magic `%%` not found.
